In [66]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import GridSearchCV

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

from bs4 import BeautifulSoup


In [42]:
df = pd.read_csv('./incident.csv', encoding='latin')

In [6]:
df

,u_product,assignment_group,short_description
0,Ariba (DRR) (703440),ITOPS-FIT-TTR-L2,IMPORTED RECEIPTS IN ARIBA FROM SAP!
1,SonicWall(DRR) (422697),ITOPS-DISD-L1,hi
2,Ariba (DRR) (703440),ITOPS-CORP-SUPPORT,Jabra Headset not functioning
3,Ariba (DRR) (703440),ITOPS-CORP-SUPPORT,I have a plantronics headset and it is not wor...
4,SonicWall(DRR) (422697),ITOPS-DISD-L1,Couldnot connect to VPN
5,SonicWall(DRR) (422697),ITOPS-DISD-L1,unable to access vpn
6,Corporate Extended Oracle WebCenter (CEOWC) (4...,ITOPS-CORP-L2,Paybridge OWC Extract is not working
7,Ariba (DRR) (703440),ITOPS-CORP-SUPPORT,[IP] Request submission in Ariba
8,SonicWall(DRR) (422697),ITOPS-DISD-L1,VPN problem
9,Ariba (DRR) (703440),ITOPS-CORP-SUPPORT,Ariba access


In [8]:
z =df['assignment_group'].value_counts()

In [58]:
df['level'] = df['assignment_group'].apply(lambda x: x.split('-')[-1])

In [61]:
df.level.value_counts()

CORP                            3984
L2                              1496
SUPPORT                          381
L1                               242
IEO ITRC Monitoring Server       141
SALES                             88
GENEVA                            27
OPE                               27
IEO ITRC Monitoring Database      21
SERVER                            19
MGMT                              17
SERVICES                          16
OM                                14
DUBAI                             14
 DEVELOPMENT                      13
BUENOSAIRES                       12
GSD                               10
SOLUTION                          10
AUSTIN                             8
PARIS                              7
SHIP                               7
OKLAHOMACITY                       7
OPERATIONS                         7
DATABASE                           5
BURN                               5
BANGALORE                          5
L3                                 3
C

In [37]:
df['u_product'].value_counts()

Corporate Oracle WebCenter (COWC) (379227)                 2835
Corporate Extended Oracle WebCenter (CEOWC) (417127)       1813
BQMS - BRAZIL QUALITY MANAGEMENT SYSTEM (35998)             995
Ariba (DRR) (703440)                                        343
HRDM - Human Resources Document Management (32300)          306
SonicWall(DRR) (422697)                                     235
EDMS - ELECTRONIC DOCUMENT MANAGEMENT SYSTEM (35994)         48
ES - EMPLOYMENT SERVICES (35995)                             37
MARS (Marketing and Advertising Review System) (250250)      17
Gale(DRR) (422694)                                            7
Performance Link (DRR) (883344)                               2
Softheon (DRR) (883348)                                       2
Force 10(DRR) (422695)                                        2
GOE - GLOBAL OPERATIONS ENGINEERING (35997)                   2
Name: u_product, dtype: int64

In [22]:
zz = set()

In [44]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [45]:
# Combining all the above stundents 
from tqdm import tqdm
short_desc = []
# tqdm is for printing the status bar
for sentance in tqdm(df['short_description'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    short_desc.append(sentance.strip())

100%|██████████| 6644/6644 [00:02<00:00, 3310.91it/s]


In [50]:
df['cleaned'] = short_desc

In [76]:
X_train, X_test, Y_train, Y_test = train_test_split(df['cleaned'], df['u_product'],
                                                    shuffle=False, test_size=0.2)

In [77]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
tf_idf_vect.fit(X_train)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

X_train_tf_idf = tf_idf_vect.transform(X_train)
print("the type of count vectorizer ",type(X_train_tf_idf))
print("the shape of out text TFIDF vectorizer ",X_train_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", X_train_tf_idf.get_shape()[1])

some sample features(unique words in the corpus) ['able', 'able access', 'able approve', 'able connect', 'able connecting', 'able create', 'able log', 'able login', 'able push', 'able raise']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (5315, 3783)
the number of unique words including both unigrams and bigrams  3783


In [78]:
X_test_tfidf = tf_idf_vect.transform(X_test)

In [79]:
X_test_tfidf

<1329x3783 sparse matrix of type '<class 'numpy.float64'>'
	with 16121 stored elements in Compressed Sparse Row format>

In [80]:
from xgboost import XGBClassifier

In [81]:
clf = XGBClassifier()
clf.fit(X_train_tf_idf, Y_train)
clf.score(X_test_tfidf, Y_test)

0.6117381489841986

In [82]:
params = {
    'n_estimators': [i for i in range(40, 150, 20)],
    'max_depth': [i for i in range(10, 50, 10)]
}


optimized_GBM = GridSearchCV(XGBClassifier(), params, cv = 5, n_jobs = 4, verbose=True)

In [83]:
optimized_GBM.fit(X_train_tf_idf, Y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:  5.9min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'n_estimators': [40, 60, 80, 100, 120, 140], 'max_depth': [10, 20, 30, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [84]:
optimized_GBM.score(X_test_tfidf, Y_test)

0.6155003762227238

In [85]:
df

,u_product,assignment_group,short_description,cleaned,level
0,Ariba (DRR) (703440),ITOPS-FIT-TTR-L2,IMPORTED RECEIPTS IN ARIBA FROM SAP!,imported receipts ariba sap,L2
1,SonicWall(DRR) (422697),ITOPS-DISD-L1,hi,hi,L1
2,Ariba (DRR) (703440),ITOPS-CORP-SUPPORT,Jabra Headset not functioning,jabra headset not functioning,SUPPORT
3,Ariba (DRR) (703440),ITOPS-CORP-SUPPORT,I have a plantronics headset and it is not wor...,plantronics headset not working office phone,SUPPORT
4,SonicWall(DRR) (422697),ITOPS-DISD-L1,Couldnot connect to VPN,couldnot connect vpn,L1
5,SonicWall(DRR) (422697),ITOPS-DISD-L1,unable to access vpn,unable access vpn,L1
6,Corporate Extended Oracle WebCenter (CEOWC) (4...,ITOPS-CORP-L2,Paybridge OWC Extract is not working,paybridge owc extract not working,L2
7,Ariba (DRR) (703440),ITOPS-CORP-SUPPORT,[IP] Request submission in Ariba,ip request submission ariba,SUPPORT
8,SonicWall(DRR) (422697),ITOPS-DISD-L1,VPN problem,vpn problem,L1
9,Ariba (DRR) (703440),ITOPS-CORP-SUPPORT,Ariba access,ariba access,SUPPORT


In [87]:
X_train, X_test, Y_train, Y_test = train_test_split(df['cleaned'], df['level'],
                                                    shuffle=False, test_size=0.2)

In [88]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
tf_idf_vect.fit(X_train)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

X_train_tf_idf = tf_idf_vect.transform(X_train)
print("the type of count vectorizer ",type(X_train_tf_idf))
print("the shape of out text TFIDF vectorizer ",X_train_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", X_train_tf_idf.get_shape()[1])

X_test_tfidf = tf_idf_vect.transform(X_test)

some sample features(unique words in the corpus) ['able', 'able access', 'able approve', 'able connect', 'able connecting', 'able create', 'able log', 'able login', 'able push', 'able raise']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (5315, 3783)
the number of unique words including both unigrams and bigrams  3783


In [91]:
clf = XGBClassifier()
clf.fit(X_train_tf_idf, Y_train)
clf.score(X_test_tfidf, Y_test)

0.7863054928517682

In [93]:
list(df.iterrows())

[(0, u_product                            Ariba (DRR) (703440)
  assignment_group                         ITOPS-FIT-TTR-L2
  short_description    IMPORTED RECEIPTS IN ARIBA FROM SAP!
  cleaned                       imported receipts ariba sap
  level                                                  L2
  Name: 0, dtype: object), (1, u_product            SonicWall(DRR) (422697)
  assignment_group               ITOPS-DISD-L1
  short_description                         hi
  cleaned                                   hi
  level                                     L1
  Name: 1, dtype: object), (2,
  u_product                     Ariba (DRR) (703440)
  assignment_group                ITOPS-CORP-SUPPORT
  short_description    Jabra Headset not functioning
  cleaned              jabra headset not functioning
  level                                      SUPPORT
  Name: 2, dtype: object), (3,
  u_product                                         Ariba (DRR) (703440)
  assignment_group               